**Note:**

This version does not currently integrate proxy servers into the webscraping process, which can cause issues when scraping from certain locations, in particular Github Actions. The alternative version of this notebook, (09a), uses a webscraping service, ScrapingAnt, which handles all the proxy server issues, but does require an account.

In [1]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

from src.webscraping_b import (
    activate_web_driver,
    scrape_to_dataframe,
    convert_columns,
    combine_home_visitor,  
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)

import json

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')

**Load API keys**

In [2]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

**Activate Webdriver**

In [3]:
# initiate a webdriver in selenium 
# since website data is dynamically generated

driver = activate_web_driver('chromium')

2023-01-29 10:48:20,947 INFO: ====== WebDriver manager ======
2023-01-29 10:48:21,231 INFO: Get LATEST chromedriver version for chromium None
2023-01-29 10:48:22,503 INFO: Driver [C:\Users\Chris\.wdm\drivers\chromedriver\win32\109.0.5414\chromedriver.exe] found in cache


**Scrape New Completed Games and Format Them**

In [4]:

def get_new_games(driver)-> pd.DataFrame:

    # set search for yesterday's games
    DAYS = 5
    SEASON = "" #no season will cause website to default to current season, format is "2022-23"
    TODAY = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    LASTWEEK = (TODAY - timedelta(days=DAYS))
    DATETO = TODAY.strftime("%m/%d/%y")
    DATEFROM = LASTWEEK.strftime("%m/%d/%y")


    df = scrape_to_dataframe(driver, Season=SEASON, DateFrom=DATEFROM, DateTo=DATETO)

    df = convert_columns(df)

    print(df.info())
    df = combine_home_visitor(df)

    return df

df_new = get_new_games(driver)

# get the SEASON of the last game in the database
# this will used when constructing rows for prediction
SEASON = df_new['SEASON'].max()

df_new




https://www.nba.com/stats/teams/boxscores?DateFrom=01/24/23&DateTo=01/29/23
all pages loaded
<class 'pandas.core.frame.DataFrame'>
Int64Index: 78 entries, 0 to 77
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   HOME            78 non-null     int64         
 1   GAME_DATE_EST   78 non-null     datetime64[ns]
 2   HOME_TEAM_WINS  78 non-null     int64         
 3   PTS             78 non-null     int64         
 4   FG_PCT          78 non-null     float64       
 5   FG3_PCT         78 non-null     float64       
 6   FT_PCT          78 non-null     float64       
 7   REB             78 non-null     int64         
 8   AST             78 non-null     int64         
 9   TEAM_ID         78 non-null     object        
 10  GAME_ID         78 non-null     object        
dtypes: datetime64[ns](1), float64(3), int64(5), object(2)
memory usage: 7.3+ KB
None


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2023-01-28,0,121,44.8,28.6,75.0,47,30,1610612747,22200749,125,46.4,32.5,87.2,46,18,1610612738,2022
1,2023-01-28,0,100,43.8,39.1,70.6,31,26,1610612742,22200750,108,47.6,36.1,70.8,49,26,1610612762,2022
2,2023-01-28,1,128,48.5,33.3,76.9,52,31,1610612756,22200748,118,44.0,34.5,70.6,53,32,1610612759,2022
3,2023-01-28,0,110,45.1,27.3,75.0,36,23,1610612758,22200746,117,52.5,45.9,76.2,44,25,1610612750,2022
4,2023-01-28,1,113,45.5,30.6,75.9,53,18,1610612764,22200747,103,42.4,27.6,89.5,42,21,1610612740,2022
5,2023-01-28,1,123,52.2,33.3,94.4,45,24,1610612761,22200751,105,52.6,29.2,78.3,27,18,1610612757,2022
6,2023-01-28,1,117,47.8,29.6,77.8,48,20,1610612745,22200743,114,45.8,36.8,77.4,35,23,1610612765,2022
7,2023-01-28,1,120,52.5,48.7,89.5,37,25,1610612746,22200745,113,47.4,25.9,80.0,47,24,1610612737,2022
8,2023-01-28,0,119,54.3,41.2,85.0,36,28,1610612743,22200741,126,51.7,51.4,66.7,39,31,1610612755,2022
9,2023-01-28,0,115,51.1,35.7,78.9,45,19,1610612752,22200742,122,52.9,55.0,57.1,35,31,1610612751,2022


**Retrieve todays games**

In [5]:
#retrieve list of teams playing today

# get today's games on NBA schedule
matchups, game_ids = get_todays_matchups(driver)


print(matchups)
print(game_ids)


[['1610612748', '1610612766'], ['1610612754', '1610612763'], ['1610612746', '1610612739'], ['1610612740', '1610612749']]
['22200752', '22200753', '22200754', '22200755']


**Close Webdriver**

In [6]:
driver.close() 

**Create Rows for Today's Games with Empty Stats**

In [7]:
# append today's matchups to the new games dataframe


df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
for i, matchup in enumerate(matchups):
    game_details = {'HOME_TEAM_ID': matchup[1], 
                    'VISITOR_TEAM_ID': matchup[0], 
                    'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                    'GAME_ID': int(game_ids[i]),                       
                    'SEASON': SEASON,
                    } 
    game_details_df = pd.DataFrame(game_details, index=[i])
    # append to new games dataframe
    df_today = pd.concat([df_today, game_details_df], ignore_index = True)

#blank rows will be filled with 0 to prevent issues with feature engineering
df_today = df_today.fillna(0) 

df_today


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2023-01-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612766,22200752,0.0,0.0,0.0,0.0,0.0,0.0,1610612748,2022
1,2023-01-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612763,22200753,0.0,0.0,0.0,0.0,0.0,0.0,1610612754,2022
2,2023-01-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612739,22200754,0.0,0.0,0.0,0.0,0.0,0.0,1610612746,2022
3,2023-01-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612749,22200755,0.0,0.0,0.0,0.0,0.0,0.0,1610612740,2022


**Access Feature Store**

In [8]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3350


Connected. Call `.close()` to terminate connection gracefully.


**Access Feature Group**

In [9]:
rolling_stats_fg = fs.get_feature_group(
    name="rolling_stats",
    version=1,
)

**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [10]:
BASE_FEATURES = ['game_date_est',
 'game_id',
 'home_team_id',
 'visitor_team_id',
 'season',
 'pts_home',
 'fg_pct_home',
 'ft_pct_home',
 'fg3_pct_home',
 'ast_home',
 'reb_home',
 'pts_away',
 'fg_pct_away',
 'ft_pct_away',
 'fg3_pct_away',
 'ast_away',
 'reb_away',
 'home_team_wins',
]

ds_query = rolling_stats_fg.select(BASE_FEATURES)
df_old = ds_query.read()
df_old


2023-01-29 10:49:54,245 INFO: USE `nba_predictor_featurestore`
2023-01-29 10:49:55,221 INFO: SELECT `fg0`.`game_date_est` `game_date_est`, `fg0`.`game_id` `game_id`, `fg0`.`home_team_id` `home_team_id`, `fg0`.`visitor_team_id` `visitor_team_id`, `fg0`.`season` `season`, `fg0`.`pts_home` `pts_home`, `fg0`.`fg_pct_home` `fg_pct_home`, `fg0`.`ft_pct_home` `ft_pct_home`, `fg0`.`fg3_pct_home` `fg3_pct_home`, `fg0`.`ast_home` `ast_home`, `fg0`.`reb_home` `reb_home`, `fg0`.`pts_away` `pts_away`, `fg0`.`fg_pct_away` `fg_pct_away`, `fg0`.`ft_pct_away` `ft_pct_away`, `fg0`.`fg3_pct_away` `fg3_pct_away`, `fg0`.`ast_away` `ast_away`, `fg0`.`reb_away` `reb_away`, `fg0`.`home_team_wins` `home_team_wins`
FROM `nba_predictor_featurestore`.`rolling_stats_1` `fg0`


,game_date_est,game_id,home_team_id,visitor_team_id,season,pts_home,fg_pct_home,ft_pct_home,fg3_pct_home,ast_home,reb_home,pts_away,fg_pct_away,ft_pct_away,fg3_pct_away,ast_away,reb_away,home_team_wins
0,2007-02-25,20600832,1610612765,1610612741,2006,95,0.377930,0.799805,0.500000,20,41,93,0.479980,0.666992,0.385010,21,45,1
1,2006-12-11,20600301,1610612753,1610612756,2006,89,0.451904,0.523926,0.166992,18,42,103,0.514160,0.852051,0.250000,25,36,0
2,2007-11-10,20700084,1610612758,1610612750,2007,100,0.405029,0.850098,0.399902,12,38,93,0.438965,0.750000,0.213989,15,43,1
3,2005-01-11,20400504,1610612760,1610612746,2004,104,0.437988,0.889160,0.500000,24,40,99,0.438965,0.758789,0.333008,23,47,1
4,2016-10-29,21600028,1610612752,1610612763,2016,111,0.525879,0.641113,0.285889,24,41,104,0.429932,0.808105,0.360107,19,40,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23513,2005-02-12,20400743,1610612765,1610612764,2004,107,0.532227,0.738770,0.500000,33,42,86,0.363037,0.666992,0.399902,19,46,1
23514,2018-10-29,21800092,1610612749,1610612761,2018,124,0.489014,0.789062,0.422119,28,57,109,0.378906,0.786133,0.199951,25,51,1
23515,2011-01-24,21000653,1610612751,1610612739,2010,103,0.446045,0.758789,0.538086,24,40,101,0.451904,0.833008,0.356934,18,41,1
23516,2017-03-06,21600934,1610612755,1610612749,2016,98,0.437988,0.631836,0.267090,30,42,112,0.493896,0.799805,0.424072,28,41,0


**Convert Feature Names back to original mixed case**

In [11]:
df_old = convert_feature_names(df_old)
df_old
df_old[df_old['PTS_home'] == 0]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS


**Update Yesterday's Matchup Predictions with New Final Results**

In [12]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results

#df_old = df_old.set_index('GAME_ID')
#df_old.update(df_new.set_index('GAME_ID'))
#df_old = df_old.reset_index()  

# another approach is to simply drop the rows that were used for prediction yesterday
# which are games that have 0 points for home team
# and then append the new rows to the dataframe
df_old = df_old[df_old['PTS_home'] != 0]
df_old = pd.concat([df_old, df_new], ignore_index = True)

df_old

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2007-02-25,20600832,1610612765,1610612741,2006,95,0.377930,0.799805,0.500000,20,41,93,0.479980,0.666992,0.385010,21,45,1
1,2006-12-11,20600301,1610612753,1610612756,2006,89,0.451904,0.523926,0.166992,18,42,103,0.514160,0.852051,0.250000,25,36,0
2,2007-11-10,20700084,1610612758,1610612750,2007,100,0.405029,0.850098,0.399902,12,38,93,0.438965,0.750000,0.213989,15,43,1
3,2005-01-11,20400504,1610612760,1610612746,2004,104,0.437988,0.889160,0.500000,24,40,99,0.438965,0.758789,0.333008,23,47,1
4,2016-10-29,21600028,1610612752,1610612763,2016,111,0.525879,0.641113,0.285889,24,41,104,0.429932,0.808105,0.360107,19,40,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23552,2023-01-24,22200714,1610612738,1610612748,2022,95,43.200000,77.800000,31.300000,21,44,98,36.200000,78.300000,36.400000,21,51,0
23553,2023-01-24,22200713,1610612741,1610612754,2022,110,48.900000,76.000000,20.800000,19,45,116,42.400000,78.600000,27.000000,24,45,0
23554,2023-01-24,22200718,1610612766,1610612756,2022,97,40.600000,72.200000,24.000000,19,40,128,50.000000,92.300000,56.400000,38,55,0
23555,2023-01-24,22200717,1610612764,1610612742,2022,127,50.600000,78.000000,36.700000,28,41,126,52.800000,84.600000,33.300000,24,38,1


**Add Today's Matchups for Feature Engineering**

In [13]:
df_combined = pd.concat([df_old, df_today], ignore_index = True)
df_combined

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2007-02-25 00:00:00,20600832,1610612765,1610612741,2006,95.0,0.377930,0.799805,0.500000,20.0,41.0,93.0,0.479980,0.666992,0.385010,21.0,45.0,1.0
1,2006-12-11 00:00:00,20600301,1610612753,1610612756,2006,89.0,0.451904,0.523926,0.166992,18.0,42.0,103.0,0.514160,0.852051,0.250000,25.0,36.0,0.0
2,2007-11-10 00:00:00,20700084,1610612758,1610612750,2007,100.0,0.405029,0.850098,0.399902,12.0,38.0,93.0,0.438965,0.750000,0.213989,15.0,43.0,1.0
3,2005-01-11 00:00:00,20400504,1610612760,1610612746,2004,104.0,0.437988,0.889160,0.500000,24.0,40.0,99.0,0.438965,0.758789,0.333008,23.0,47.0,1.0
4,2016-10-29 00:00:00,21600028,1610612752,1610612763,2016,111.0,0.525879,0.641113,0.285889,24.0,41.0,104.0,0.429932,0.808105,0.360107,19.0,40.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23556,2023-01-24 00:00:00,22200716,1610612743,1610612740,2022,99.0,48.800000,76.900000,33.300000,27.0,34.0,98.0,44.700000,85.000000,19.200000,18.0,43.0,1.0
23557,2023-01-29,22200752,1610612766,1610612748,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
23558,2023-01-29,22200753,1610612763,1610612754,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
23559,2023-01-29,22200754,1610612739,1610612746,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0


**Data Processing**

In [14]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PLAYOFF,TARGET
0,2007-02-25 00:00:00,20600832,1610612765,1610612741,2006,95.0,0.377930,0.799805,0.500000,20.0,41.0,93.0,0.479980,0.666992,0.385010,21.0,45.0,1.0,0,1.0
1,2006-12-11 00:00:00,20600301,1610612753,1610612756,2006,89.0,0.451904,0.523926,0.166992,18.0,42.0,103.0,0.514160,0.852051,0.250000,25.0,36.0,0.0,0,0.0
2,2007-11-10 00:00:00,20700084,1610612758,1610612750,2007,100.0,0.405029,0.850098,0.399902,12.0,38.0,93.0,0.438965,0.750000,0.213989,15.0,43.0,1.0,0,1.0
3,2005-01-11 00:00:00,20400504,1610612760,1610612746,2004,104.0,0.437988,0.889160,0.500000,24.0,40.0,99.0,0.438965,0.758789,0.333008,23.0,47.0,1.0,0,1.0
4,2016-10-29 00:00:00,21600028,1610612752,1610612763,2016,111.0,0.525879,0.641113,0.285889,24.0,41.0,104.0,0.429932,0.808105,0.360107,19.0,40.0,1.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23517,2011-04-11 00:00:00,21001203,1610612751,1610612766,2010,103.0,0.543945,0.929199,0.399902,23.0,29.0,105.0,0.569824,0.687988,0.308105,27.0,30.0,0.0,0,0.0
23557,2023-01-29,22200752,1610612766,1610612748,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0
23558,2023-01-29,22200753,1610612763,1610612754,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0
23559,2023-01-29,22200754,1610612739,1610612746,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0


**Feature Engineering**

In [15]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)
df_combined


#fix type conversion issues with hopsworks
df_combined['TARGET'] = df_combined['TARGET'].astype('int16')
df_combined['HOME_TEAM_WINS'] = df_combined['HOME_TEAM_WINS'].astype('int16')


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

**Insert New Data into Feature Group**

In [16]:

def test():
    # retrieve only new games from the combined dataframe now that feature engineering is complete

    # set index to GAME_ID
    df_combined = df_combined.set_index('GAME_ID')
    df_new = df_new.set_index('GAME_ID')
    
    # retrieve only new games
    df_new = df_combined.loc[df_new.index]

    # reset GAME_ID index back to column
    df_new = df_new.reset_index()

    # convert certain features back to int32 for Hopsworks compatibility
    df_new['GAME_ID'] = df_new['GAME_ID'].astype('int32')
    df_new['HOME_TEAM_WINS'] = df_new['HOME_TEAM_WINS'].astype('int32')
    df_new['TARGET'] = df_new['TARGET'].astype('int32')

    # save new games to Hopsworks feature group
    rolling_stats_fg.insert(df_new, write_options={"wait_for_job" : False})

    df_new 

rolling_stats_fg.insert(df_combined, write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/23522 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3350/jobs/named/rolling_stats_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x16ced0da340>, None)

In [17]:
df_old.tail(20)

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
23537,2023-01-26,22200732,1610612741,1610612766,2022,96,40.7,81.8,16.0,16,43,111,46.4,83.9,21.9,24,52,0
23538,2023-01-26,22200731,1610612765,1610612751,2022,130,50.5,88.5,40.7,27,48,122,45.7,85.2,34.2,24,40,1
23539,2023-01-26,22200730,1610612752,1610612738,2022,120,45.8,82.6,34.2,17,50,117,41.0,82.6,34.8,22,54,1
23540,2023-01-25,22200725,1610612737,1610612760,2022,137,59.5,78.8,37.9,26,46,132,52.0,68.8,44.1,31,37,1
23541,2023-01-25,22200729,1610612759,1610612747,2022,104,37.7,88.9,23.5,22,48,113,44.4,76.7,30.3,25,56,0
23542,2023-01-25,22200728,1610612761,1610612758,2022,113,49.5,71.4,30.0,28,41,95,47.9,92.3,40.6,29,34,1
23543,2023-01-25,22200727,1610612762,1610612757,2022,124,52.2,66.7,48.8,32,35,134,57.3,83.3,47.2,26,41,0
23544,2023-01-25,22200720,1610612754,1610612753,2022,120,48.2,79.3,42.9,25,32,126,56.1,76.0,53.6,34,34,0
23545,2023-01-25,22200724,1610612750,1610612740,2022,111,48.8,71.4,37.9,23,36,102,43.5,76.2,31.6,25,42,1
23546,2023-01-25,22200723,1610612743,1610612749,2022,99,39.2,75.0,32.4,21,51,107,47.4,71.4,43.3,16,49,0


In [18]:
df_combined[df_combined['GAME_ID'] > 22200640]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
23407,2023-01-14,22200641,1610612749,1610612748,2022,95,40.50000,66.6875,39.59375,26,...,1.879687,0.347917,4.333333,1.857143,1.0,0.333333,0.333333,6.714286,10.2,9.200000
23408,2023-01-14,22200645,1610612739,1610612750,2022,102,48.68750,87.0000,28.59375,24,...,-1.815625,-1.963542,2.000000,-0.142857,0.9,-0.066667,1.000000,4.000000,2.5,1.666667
23409,2023-01-14,22200646,1610612755,1610612762,2022,118,44.90625,80.8125,43.59375,23,...,4.151563,2.839583,8.333333,2.714286,1.4,2.733333,-7.000000,-2.857143,-5.2,-1.533333
23410,2023-01-14,22200644,1610612737,1610612761,2022,114,51.18750,94.3750,40.68750,19,...,-0.345312,2.051042,-5.000000,-1.142857,0.0,1.466667,4.000000,4.142857,3.5,2.666667
23411,2023-01-14,22200643,1610612763,1610612754,2022,130,55.18750,78.6250,37.50000,32,...,0.153125,-2.714583,-1.666667,-1.000000,-1.2,-0.866667,4.333333,8.285714,7.8,8.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23517,2023-01-28,22200745,1610612746,1610612737,2022,120,52.50000,89.5000,48.68750,25,...,1.846875,2.395833,-1.000000,-1.285714,-1.7,-1.466667,0.333333,2.571429,3.2,0.200000
23518,2023-01-29,22200754,1610612739,1610612746,2022,0,0.00000,0.0000,0.00000,0,...,-9.393750,-5.950000,3.000000,3.285714,3.2,2.666667,-2.333333,-0.285714,-1.4,-0.333333
23519,2023-01-29,22200753,1610612763,1610612754,2022,0,0.00000,0.0000,0.00000,0,...,4.342188,1.509375,1.666667,1.571429,1.1,0.666667,-0.333333,5.000000,4.6,5.933333
23520,2023-01-29,22200752,1610612766,1610612748,2022,0,0.00000,0.0000,0.00000,0,...,-0.548438,-0.546875,-0.333333,0.285714,2.5,3.266667,1.333333,-1.857143,-3.1,0.333333


In [19]:
df_combined[df_combined['PTS_home'] == 0]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
23518,2023-01-29,22200754,1610612739,1610612746,2022,0,0.0,0.0,0.0,0,...,-9.393750,-5.950000,3.000000,3.285714,3.2,2.666667,-2.333333,-0.285714,-1.4,-0.333333
23519,2023-01-29,22200753,1610612763,1610612754,2022,0,0.0,0.0,0.0,0,...,4.342188,1.509375,1.666667,1.571429,1.1,0.666667,-0.333333,5.000000,4.6,5.933333
23520,2023-01-29,22200752,1610612766,1610612748,2022,0,0.0,0.0,0.0,0,...,-0.548438,-0.546875,-0.333333,0.285714,2.5,3.266667,1.333333,-1.857143,-3.1,0.333333
23521,2023-01-29,22200755,1610612749,1610612740,2022,0,0.0,0.0,0.0,0,...,9.381250,6.579167,7.333333,3.142857,2.2,2.333333,7.333333,3.857143,4.6,5.600000
